# Task- 3: Multilabel Classification of Counter Speech
### This notebook is used to measure performance of the combination of different classifier and different feature engineering techniques  used in the project

In [3]:
from commen_preprocess import *
from skmultilearn.adapt import brknn
from skmultilearn.adapt import MLkNN
from skmultilearn.problem_transform import br
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.neurofuzzy import MLARAM
import scipy
import sklearn.metrics
import argparse
import numpy
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn import neural_network
from sklearn import linear_model
import gensim, sklearn
from collections import defaultdict
#from batch_gen import batch_gen
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from xgboost import XGBClassifier
from sklearn.externals import joblib
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import VotingClassifier
import os
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
import tensorflow as tf
import tensorflow_hub as hub
import re
from tqdm import tqdm
import pickle
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix,make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
ps = PorterStemmer()
import pandas as pd
import numpy as np
import json
from catboost import CatBoostClassifier
from scipy.sparse import vstack, hstack



/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Word_to_vec model  loading 
1. change the path of glove model file
2. One the given function must load the model

In [25]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import os
#GLOVE_MODEL_FILE_200 = "../../../LEAM-master/glove.twitter.27B/glove.twitter.27B.200d.txt"
GLOVE_MODEL_FILE="../../glove.840B.300d.txt"
print(os.path.isfile(GLOVE_MODEL_FILE))

import numpy as np

## change the embedding dimension according to the model
EMBEDDING_DIM = 300
###change the method type
select_method=1

### method one
def loadGloveModel1(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    i=0
    for line in f:
        i=i+1
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
        if(i%10000==0):
            print("count"+str(i/10000))
    print("Done.",len(model)," words loaded!")
    return model


### method two
def loadGloveModel2(glove_file):
    tmp_file = get_tmpfile("test_crawl_200.txt")

    # call glove2word2vec script
    # default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>

    glove2word2vec(glove_file, tmp_file)
    model=KeyedVectors.load_word2vec_format(tmp_file)
    return model


if (select_method==1):
    word2vec_model = loadGloveModel1(GLOVE_MODEL_FILE)
elif (select_method==2):
    word2vec_model = loadGloveModel2(GLOVE_MODEL_FILE)
else:
    print("wrong method number selected")


True
Loading Glove Model
count1.0
count2.0
count3.0
count4.0
count5.0
count6.0
count7.0
count8.0
count9.0
count10.0
count11.0
count12.0
count13.0
count14.0
count15.0
count16.0
count17.0
count18.0
count19.0
count20.0
count21.0
count22.0
count23.0
count24.0
count25.0
count26.0
count27.0
count28.0
count29.0
count30.0
count31.0
count32.0
count33.0
count34.0
count35.0
count36.0
count37.0
count38.0
count39.0
count40.0
count41.0
count42.0
count43.0
count44.0
count45.0
count46.0
count47.0
count48.0
count49.0
count50.0
count51.0
count52.0
count53.0
count54.0
count55.0
count56.0
count57.0
count58.0
count59.0
count60.0
count61.0
count62.0
count63.0
count64.0
count65.0
count66.0
count67.0
count68.0
count69.0
count70.0
count71.0
count72.0
count73.0
count74.0
count75.0
count76.0
count77.0
count78.0
count79.0
count80.0
count81.0
count82.0
count83.0
count84.0
count85.0
count86.0
count87.0
count88.0
count89.0
count90.0
count91.0
count92.0
count93.0
count94.0
count95.0
count96.0
count97.0
count98.0
coun

#### This function is generating the classification report
1. input: ground_truth and predicted outputs
2. output: dataframe containing the results

In [26]:
##saving the classification report
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

### Google Universal Sentence Encoder configuration
###### prerequisite: tensorflow version >=1.7

In [27]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=12,
                       allow_soft_placement=True, device_count = {'CPU': 12})

def get_embeddings(messages):
      
    with tf.Session(config=config) as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_emb = session.run(embed(messages))
            
    print("ending")
    return np.array(message_emb)

INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_12:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_12

### Dataset is loaded here

In [28]:
### set the path of the the json file where to load the data 
path='../Counter_NonCounter/Data/Counterspeech_Dataset.json'
with open(path) as fp:
    train_data = json.load(fp)

In [29]:
### return label
def convert_class_label(input_text):
    if input_text:
        return 'counter'
    else:
        return 'noncounter'

#### Load the data in the dataframe  having the four fields as
1. id
2. class
3. community
4. category(labels)
5. text

In [30]:
pd_train = pd.DataFrame(columns=['id','class','community','category','text'])

for count, each in enumerate(train_data):
    try:
        pd_train.loc[count]  = [each['id'], convert_class_label(each['CounterSpeech']), each['Community'],each['Category'],each['commentText']]
    except:
        pass
print('Training Data Loading Completed...') 

### select the data having the labels ...default denotes the non counter speech
pd_train =pd_train[pd_train['category']!='Default']

Training Data Loading Completed...


#### Extraction of the category column into a multi-hot vector encoding 

In [31]:
list1=[[],[],[],[],[],[],[],[],[],[]]
for ele in pd_train['category']:
    temp=[]
    #print(ele)
    for i in range(0,len(ele),2):
        temp.append(ord(ele[i])-ord('0'))
    #print(temp)
    if(len(temp)==0):
        print(temp)
    for i in range(0,10):
        if i+1 in temp:
            list1[i].append(1)
        else:
            list1[i].append(0)
y_train=np.array([np.array(xi) for xi in list1])            

In [32]:
### final dataframe for the task created 
pd_train = pd.DataFrame({'text':list(pd_train['text']),'cat0':list1[0],'cat1':list1[1],'cat2':list1[2],'cat3':list1[3],'cat4':list1[4],'cat5':list1[5],'cat6':list1[6],'cat7':list1[7],'cat8':list1[8],'cat9':list1[9]})
### drop the entries having blank entries
pd_train['text'].replace('', np.nan, inplace=True)
pd_train.dropna(subset=['text'], inplace=True)


In [33]:
pd_train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,text
0,0,1,0,0,0,0,0,1,0,0,What kind of God is it that Hates people & Cur...
1,0,0,0,0,0,0,0,1,0,0,Question - Which comment is racist and why?\n ...
2,0,0,0,0,0,0,0,1,0,0,That Israel fail that is nothing new. That Isr...
3,0,0,0,0,0,0,0,0,1,0,This god damned ignorant little prick doesn't ...
4,0,1,0,0,0,0,0,0,0,0,our shit may be cooler but... police brutality...


In [34]:
#### converting the data into text and labels dictionary
def get_data():
    comments=pd_train['text'].values
    df = pd_train.drop('text', 1)
    labels=df.values
    list_comment=[]
    for comment,label in zip(comments,labels):
        temp={}
        temp['text']=comment
        temp['label']=label
        list_comment.append(temp)
    return list_comment    

#### Different type of tokenization that can be performed

In [35]:
### stopwords and punctuations are not removed but text is cleaned and stemmed
def glove_tokenize_norem(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    words = text.split()
    words =[ps.stem(word) for word in words]
    return words

####stopwords and punctuations are removed along with that text is cleaned and stemmed
def glove_tokenize(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    words =[ps.stem(word) for word in words]
    return words

####stopwords and punctuations are removed along with that text is cleaned
def glove_tokenize_embed(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    words =[ps.stem(word) for word in words]
    return words

### Different feature generation methods


In [36]:
TOKENIZER = glove_tokenize
#google encoding used where text is cleaned  
def gen_data_google():
    comments = get_data()
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        #X.append(tokenizer(comment['text']))
        X.append(clean(comment['text'], remove_stopwords=True, remove_punctuations=True))
    
    X =get_embeddings(X)
    return X, y

#google encoding used where text is not cleaned 
def gen_data_google2():
    comments = get_data()
    X, X1, y = [],[],[]
    for comment in comments:
        y.append(comment['label'])
        X.append(clean(comment['text'], remove_stopwords=False, remove_punctuations=False))
    #Word Level Features
    X =get_embeddings(X)
    return X,y

### tfidf feature generation was used here where stopwords and punctuations are removed 
def gen_data_new_tfidf():
    comments = get_data()
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=False, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                stop_words='english',
                tokenizer=TOKENIZER,             
                max_features=500)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=False,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=500)
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    return X, y

### tfidf feature generation was used here where stopwords and punctuations are not removed 
def gen_data_new_tfidf2():
    comments = get_data()
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=False,ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                #stop_words='english',
                tokenizer=glove_tokenize_norem,             
                max_features=500)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=False,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=500)
    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    
    return X, y

def gen_data_embed():
    comments = get_data()
    X, y = [], []
    for comment in comments:
        words = glove_tokenize_embed(comment['text'].lower())
        emb = numpy.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    return X, y


## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_rem():
    X,_=gen_data_google()
    X1,y=gen_data_new_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

## combination of cleaned google encodings and tfidf features where stopwords and punctuations are ssremoved 
def combine_tf_google_norem():
    X,_=gen_data_google2()
    X1,y=gen_data_new_tfidf2()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y
## combination of google encodings where stopwords and punctuation are kept and tfidf features where stopwords and punctuations are removed 
def combine_tf_rem_google_norem():
    X,_=gen_data_google2() 
    X1,y=gen_data_new_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y
## combination of google encodings where stopwords and punctuation are removed and tfidf features where stopwords and punctuations are kept 
def combine_tf_norem_google_rem():
    X,_=gen_data_google()
    X1,y=gen_data_new_tfidf2()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

## combination of google encodings where stopwords and punctuation are removed and average word embeddings  
def combine_google_rem_embed():
    X,_=gen_data_google()
    X1,y=gen_data_embed()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y
## combination of tfidf features where stopwords and punctuation are removed and average word embeddings  
def combine_tf_rem_embed():
    X,_=gen_data_new_tfidf()
    X1,y=gen_data_embed()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

####combination of three
def combine_tf_rem_google_norem_embed():
    X,_=gen_data_google2()
    X1,y=gen_data_new_tfidf()
    X2,_=gen_data_embed()
    X=np.concatenate((np.array(X), np.array(X1),np.array(X2)), axis=1)
    return X,y

def combine_tf_rem_google_rem_embed():
    X,_=gen_data_google()
    X1,y=gen_data_new_tfidf()
    X2,_=gen_data_embed()
    X=np.concatenate((np.array(X), np.array(X1),np.array(X2)), axis=1)
    return X,y




###old tfidf

def gen_data_old_tfidf():
    comments = get_data()
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])
    with open('../tfidf_word_vectorizer.pk', 'rb') as fin:
        word_vectorizer = pickle.load(fin)

    with open('../tfidf_char_vectorizer.pk', 'rb') as fin:
        char_vectorizer = pickle.load(fin)


    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y


In [45]:
###this is the metric used for calculating the scores 
def calculate_score(y_true, y_pred, normalize=True, sample_weight=None):

    acc_list = []
    accuracy = []
    precision = []
    recall = []
    f1_score = []

    for i in range(y_true.shape[0]):

        set_true = set( numpy.where(y_true[i])[0] )
        set_pred = set( numpy.where(y_pred[i])[0] )
        #print(set_true)
        #print(set_pred)
        
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
            temp_acc = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
            if len(set_pred) == 0:
                temp_pre = 0
            else:
                temp_pre = len(set_true.intersection(set_pred))/float( len(set_pred) )
            temp_rec = len(set_true.intersection(set_pred))/float( len(set_true))
            temp_f1 = 2*(len(set_true.intersection(set_pred)))/float(len(set_pred) + len(set_true) )
        #print('tmp_a*: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
        accuracy.append(temp_acc)
        precision.append(temp_pre)
        recall.append(temp_rec)
        f1_score.append(temp_f1)
        
        
    mean_hamming=sklearn.metrics.hamming_loss(y_true, y_pred)
    mean_accuracy=numpy.mean(accuracy)
    mean_precision=numpy.mean(precision)
    mean_recall=numpy.mean(recall)
    mean_fscore=(2*mean_precision*mean_recall)/(mean_precision+mean_recall)
    return  mean_hamming,mean_accuracy,mean_precision,mean_recall,mean_fscore


In [46]:
def my_accuracy_score(y_train,y_train_pred):
    count=0
    for ele1,ele2 in zip(y_train,y_train_pred):
        if(list(ele1)==list(ele2)):
            count=count+1
    return count/y_train.shape[0]         

### Main Classification function

In [ ]:
import json
with open('multilabel_all_parameters.json') as f:
        parameters=json.load(f)


In [47]:
def classification_model(model_name,featureset_name,model_type=None):
    X,Y=get_feature(featureset_name)
    model=get_model(model_type)
    try:
        model_parameter=parameters[classifier_model+'+'feature_model]
        for k,v in param_set.items():
             setattr(model_parameter,k,v)
    except:
        pass
    
    
    if(model==None):
        return 1
    path=model_name+'_'+featureset_name
    os.makedirs(path, exist_ok=True)
    mskf = MultilabelStratifiedKFold(n_splits=10, random_state=0)
    X = numpy.array(X)
    Y = numpy.array(Y)
    ham_list=[]
    acc_list=[]
    pre_list=[]
    rec_list=[]
    f1_list=[]
    hard_metric_list=[]
    hard_train_list=[]
    y_total_preds=[] 
    y_total=[]
    count=1
    for train_index, test_index in mskf.split(X, Y):
            #print("TRAIN:", train_index, "TEST:", test_index)
            print("crossval step--",str(count))
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]    
            clf = model
            clf.fit(X_train, y_train)
            y_pred =  clf.predict(X_test)
            y_train_pred =  clf.predict(X_train)
            
            if(scipy.sparse.issparse(y_pred)):
               ham,acc,pre,rec,f1=calculate_score(y_test,y_pred.toarray())
               for ele in y_test:
                  y_total.append(ele)
               for ele in y_pred.toarray():
                  y_total_preds.append(ele)
               accuracy_train=accuracy_score(y_train,y_train_pred.toarray())
               accuracy_test=accuracy_score(y_test,y_pred.toarray())
            
            else:
               ham,acc,pre,rec,f1=calculate_score(y_test,y_pred)
               for ele in y_test:
                  y_total.append(ele)
               for ele in y_pred:
                  y_total_preds.append(ele)
               accuracy_train=my_accuracy_score(y_train,y_train_pred)
               accuracy_test=my_accuracy_score(y_test,y_pred)
             
            ham_list.append(ham)
            acc_list.append(acc)
            pre_list.append(pre)
            rec_list.append(rec)
            f1_list.append(f1)
            hard_train_list.append(accuracy_train)
            hard_metric_list.append(accuracy_test)
            count=count+1
    y_total_preds=np.array(y_total_preds) 
    y_total=np.array(y_total)
    with open('all_preds_multilabel.pkl', 'wb') as f:
        pickle.dump([y_total,y_total_preds], f)
    
    for i in range(10):
        df_result=pandas_classification_report(y_total[:,i],y_total_preds[:,i])
        df_result.to_csv(path+'/report'+str(i)+'.csv')
        
    
    f = open(path+'/final_report.txt', "w")
    f.write(model_name)    
    f.write("The hard train score is :- " + str(numpy.mean(hard_train_list)))
    f.write("The hard metric score is :- " + str(numpy.mean(hard_metric_list)))
    f.write("The accuracy is :- " + str(numpy.mean(acc_list)))
    f.write("The precision is :- " + str(numpy.mean(pre_list)))
    f.write("The recall is :- " + str(numpy.mean(rec_list)))
    f.write("The f1_score is :- " + str(numpy.mean(f1_list)))
    f.write("The hamming loss is :-" + str(numpy.mean(ham_list)))
    f.close()


In [48]:
def get_model(m_type=None):
    if not m_type:
        print("ERROR: Please specify a model type!")
        return None
    if m_type == 'decision_tree_classifier':
        logreg = tree.DecisionTreeClassifier(class_weight='balanced')
    elif m_type == 'MLPClassifier':
        logreg = neural_network.MLPClassifier((500))
    elif m_type == 'KNeighborsClassifier':
        logreg = neighbors.KNeighborsClassifier(n_neighbors = 10)
    elif m_type == 'ExtraTreeClassifier':
        logreg = tree.ExtraTreeClassifier()
    elif m_type == 'ExtraTreeClassifier_2':
        logreg = ensemble.ExtraTreesClassifier()
    elif m_type == 'Logistic_Regression':
        logreg = OneVsRestClassifier(linear_model.LogisticRegression(class_weight='balanced'))
    elif m_type == 'RandomForestClassifier':
        logreg = ensemble.RandomForestClassifier(class_weight='balanced')
    elif m_type == 'binary_relevance_GaussianNB':
        logreg = OneVsRestClassifier(GaussianNB())
    elif m_type == 'SVC':
        logreg = OneVsRestClassifier(SVC(class_weight='balanced'))
    elif m_type == 'XG_BOOST':
        ###best_model####
        logreg = OneVsRestClassifier(XGBClassifier(eval_metric='logloss',objective= 'binary:logistic', nthread=12, scale_pos_weight=5))
    elif m_type == 'Catboost':
        logreg = OneVsRestClassifier(CatBoostClassifier(iterations=100,silent=True,scale_pos_weight=5))
    elif m_type == 'MLKNN':
        logreg = MLkNN(k=8)
    elif m_type == 'MLARAM':
        logreg = MLARAM(threshold=0.05, vigilance=0.95)
    else:
        print("ERROR: Please specify a correct model")
        return None

    return logreg


def get_feature(f_type=None):
    if not f_type:
        print("ERROR: Please specify a model type!")
        return None,None
    if f_type == 'google_not_preprocess':
        X,y=gen_data_google2()
    elif f_type == 'word_to_vec_embed':
        X,y=gen_data_embed()
    elif f_type == 'google_preprocess':
        X,y=gen_data_google()
    elif f_type == 'tfidf_not_preprocess':
        X,y=gen_data_new_tfidf2()
    elif f_type == 'tfidf_preprocess':
        X,y=gen_data_new_tfidf()
    elif f_type == 'google_preprocess_tfidf_preprocess':
        X,y=combine_tf_google_rem()
    elif f_type == 'google_nopreprocess_tfidf_nopreprocess':
        X,y=combine_tf_google_norem()
    elif f_type == 'google_preprocess_tfidf_nopreprocess':
        X,y=combine_tf_norem_google_rem()
    elif f_type == 'google_nopreprocess_tfidf_preprocess':
        X,y=combine_tf_rem_google_norem()
    elif f_type == 'google_preprocess_embed':
        X,y=combine_google_rem_embed()
    elif f_type == 'tfidf_preprocess_embed':
        X,y=combine_tf_rem_embed()
    elif f_type == 'google_preprocess_tfidf_preprocess_embed':
        ###best features####
        X,y=combine_tf_rem_google_rem_embed()
    else:
        print("give correct feature selection")    
    return X,y



## Models available
1. decision_tree_classifier
2. MLPClassifier
3. KNeighborsClassifier
4. ExtraTreeClassifier
5. ExtraTreeClassifier_2
6. RandomForestClassifier
7. SVC
8. CatboostClassifier
9. XGB_classifier
10. Logistic_Regression
11. MLKNN
12. MLARAM
13. Gaussian Naive Bayes




## Feature Models available
1. google_not_preprocess
2. word_to_vec_embed
3. google_preprocess
4. tfidf_not_preprocess
5. tfidf_preprocess
6. google_preprocess_tfidf_preprocess
7. google_nopreprocess_tfidf_nopreprocess
8. google_preprocess_tfidf_nopreprocess
9. google_nopreprocess_tfidf_preprocess
10. google_preprocess_embed
11. tfidf_preprocess_embed
12. google_preprocess_tfidf_preprocess_embed


### Specify the model and the feature selection method

In [49]:
#### abbreviated model name ####
abv_name='XG_BOOST'
#### feature_selection method name####
feature_select='google_preprocess_tfidf_preprocess_embed'
#### model name ####
model_name='XG_BOOST'

### calling of the method####
classification_model(abv_name,feature_select,model_name)


ending
crossval step-- 1


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))


crossval step-- 2


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))


crossval step-- 3


...

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))


crossval step-- 9


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))


crossval step-- 10


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
